# Incorrect Column Type Prediction

In [1]:
# Preamble to run notebook in context of source package.
# NBVAL_IGNORE_OUTPUT
import sys
sys.path.insert(0, '../')

In [2]:
from IPython.core.display import display
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcdefaults()
import numpy as np
import pandas as pd

from ptype.Ptype import Ptype
from utils import plot_column_type_posterior, plot_arff_type_posterior, subsample_df

### Toy Example

In [3]:
x = ['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '90']
column = 'year'

df = pd.DataFrame(x, dtype='str', columns=[column])
df

,year
0,1991
1,1992
2,1993
3,1994
4,1995
5,1996
6,1997
7,1998
8,90


In [4]:
ptype = Ptype()

ptype.fit_schema(df)
ptype.show_schema()

,year
type,integer
normal values,"[1991, 1992, 1993, 1994, 1995, 1996, 1997, 199..."
ratio of normal values,1
missing values,[]
ratio of missing values,0
anomalous values,[]
ratio of anomalous values,0


In [5]:
ptype.cols[column]

{'series': 0    1991
1    1992
2    1993
3    1994
4    1995
5    1996
6    1997
7    1998
8      90
Name: year, dtype: object, 'p_t': {'integer': 0.9983771135690495, 'string': 3.6348530023499018e-124, 'float': 0.00010795301310722254, 'boolean': 5.98693527768057e-276, 'date-iso-8601': 0.0015149334178433454, 'date-eu': 5.98693527768057e-276, 'date-non-std-subtype': 5.98693527768057e-276, 'date-non-std': 5.98693527768057e-276}, 'p_t_canonical': {}, 'p_z': {'integer': array([[1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 2.46635079e-22]]), 'string': array

In [6]:
ptype.cols[column].reclassify('date-iso-8601')

In [7]:
ptype.show_schema()

,year
type,date-iso-8601
normal values,"[1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998]"
ratio of normal values,0.89
missing values,[]
ratio of missing values,0
anomalous values,[90]
ratio of anomalous values,0.11


In [8]:
ptype.cols[column]

{'series': 0    1991
1    1992
2    1993
3    1994
4    1995
5    1996
6    1997
7    1998
8      90
Name: year, dtype: object, 'p_t': {'integer': 0.9983771135690495, 'string': 3.6348530023499018e-124, 'float': 0.00010795301310722254, 'boolean': 5.98693527768057e-276, 'date-iso-8601': 0.0015149334178433454, 'date-eu': 5.98693527768057e-276, 'date-non-std-subtype': 5.98693527768057e-276, 'date-non-std': 5.98693527768057e-276}, 'p_t_canonical': {}, 'p_z': {'integer': array([[1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 1.98715566e-32],
       [1.00000000e+00, 0.00000000e+00, 2.46635079e-22]]), 'string': array

### Real-world Example
In this example, we use the Grub Damage dataset to analyze the relationship between grass grub numbers, irrigation and damage.

Let us simply the problem and consider the task of finding the association between the zone and GG_new columns.

In [9]:
df = pd.read_csv('../data/grub-damage.csv', encoding="ISO-8859-1",dtype='str')
df.head()

,year_zone,year,strip,pdk,damage_rankRJT,damage_rankALL,dry_or_irr,zone,GG_new
0,6f,86,3,1,1,0,D,F,low
1,6f,86,3,2,0,0,D,F,high
2,6f,86,3,3,1,1,D,F,high
3,6f,86,3,4,1,0,D,F,high
4,6f,86,3,5,0,0,D,F,low


First, we use ptype to inspect the properties of this dataset and transform it accordingly. 

In [10]:
ptype = Ptype()

schema = ptype.fit_schema(df)
ptype.show_schema()

,year_zone,year,strip,pdk,damage_rankRJT,damage_rankALL,dry_or_irr,zone,GG_new
type,string,integer,integer,integer,integer,integer,string,boolean,string
normal values,"[0c, 0f, 0m, 1c, 1f, 1m, 2c, 2f, 2m, 6c, 6f, 6...","[86, 87, 88, 89, 90, 91, 92]","[1, 10, 2, 3, 4, 5, 6, 7, 9]","[0, 1, 2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]","[B, D, O]",[F],"[average, high, low, veryhigh]"
ratio of normal values,1,1,1,1,1,1,1,0.46,1
missing values,[],[],[],[],[],[],[],[],[]
ratio of missing values,0,0,0,0,0,0,0,0,0
anomalous values,[],[],[],[],[],[],[],"[C, M]",[]
ratio of anomalous values,0,0,0,0,0,0,0,0.54,0


As you can see, ptype predicts the data type of the zone column as boolean and labels the values of C and M as anomalies. Note that we can confirm that these values are normal values using the corresponding metadata, which states "8. zone - position of paddock (F: foothills, M: midplain, C: coastal) - enumerated".

If we are not interacting with ptype, we would obtain the following data frame.

In [11]:
df_transformed = ptype.asschema(df, schema)
df_transformed

,year_zone,year,strip,pdk,damage_rankRJT,damage_rankALL,dry_or_irr,zone,GG_new
0,6f,86,3,1,1,0,D,False,low
1,6f,86,3,2,0,0,D,False,high
2,6f,86,3,3,1,1,D,False,high
3,6f,86,3,4,1,0,D,False,high
4,6f,86,3,5,0,0,D,False,low
...,...,...,...,...,...,...,...,...,...
150,2c,92,9,4,1,1,B,<NA>,average
151,2c,92,10,1,3,3,O,<NA>,high
152,2c,92,10,2,1,1,D,<NA>,average
153,2c,92,10,3,2,2,O,<NA>,average


Therefore, the Cramers V statistic between zone and GG_new columns would be undefined due to anomalous values.

In [13]:
# NBVAL_IGNORE_OUTPUT

import scipy.stats as ss

def cramers_corrected_stat(x, y):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    confusion_matrix = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))


cramers_corrected_stat(df_transformed['zone'], df_transformed['GG_new'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


nan

Let us now interact with ptype to fix its predictions for the zone column.

In [14]:
column = 'zone'
ptype.cols[column].reclassify('string')
ptype.show_schema()

,year_zone,year,strip,pdk,damage_rankRJT,damage_rankALL,dry_or_irr,zone,GG_new
type,string,integer,integer,integer,integer,integer,string,string,string
normal values,"[0c, 0f, 0m, 1c, 1f, 1m, 2c, 2f, 2m, 6c, 6f, 6...","[86, 87, 88, 89, 90, 91, 92]","[1, 10, 2, 3, 4, 5, 6, 7, 9]","[0, 1, 2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]","[B, D, O]","[C, F, M]","[average, high, low, veryhigh]"
ratio of normal values,1,1,1,1,1,1,1,1,1
missing values,[],[],[],[],[],[],[],[],[]
ratio of missing values,0,0,0,0,0,0,0,0,0
anomalous values,[],[],[],[],[],[],[],[],[]
ratio of anomalous values,0,0,0,0,0,0,0,0,0


As we can see, the column type prediction of the zone column is now correct. Moreover, the row type predictions are also updated.

In [15]:
# we use the updated schema
schema = ptype.cols
df_transformed = ptype.asschema(df, schema)
df_transformed

,year_zone,year,strip,pdk,damage_rankRJT,damage_rankALL,dry_or_irr,zone,GG_new
0,6f,86,3,1,1,0,D,F,low
1,6f,86,3,2,0,0,D,F,high
2,6f,86,3,3,1,1,D,F,high
3,6f,86,3,4,1,0,D,F,high
4,6f,86,3,5,0,0,D,F,low
...,...,...,...,...,...,...,...,...,...
150,2c,92,9,4,1,1,B,C,average
151,2c,92,10,1,3,3,O,C,high
152,2c,92,10,2,1,1,D,C,average
153,2c,92,10,3,2,2,O,C,average


We can now calculate the Cramers V statistic as below:

In [16]:
cramers_corrected_stat(df_transformed['zone'], df_transformed['GG_new'])

0.3074039662588285